In [8]:
import gensim
import numpy as np 
import pandas as pd

In [25]:
from collections import Counter

In [45]:
from nltk.tokenize import RegexpTokenizer

In [4]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [53]:
import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
# print(lemmatizer.lemmatize("cats"))

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

def parse_reddit_csv(filename):
    print("Reading from", filename)
    csv_cols = []
    frequencies = {}
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            #remove numbers 
            row['selftext'] = re.sub(r'\d+', '', row['selftext'])
            row['title'] = re.sub(r'\d+', '', row['title'])
            # Tokenize the post text (selftext) and post title
            #remove punctuations 
            post_tokens = tokenizer.tokenize(row['selftext'])
            title_tokens = tokenizer.tokenize(row['title'])
            # Filter out stopwords
            post_tokens = [w for w in post_tokens if not w in stop_words]
            title_tokens = [w for w in title_tokens if not w in stop_words]
            # Lemmatize the post text (reduce words to word stems i.e. cats->cat, liked->like)
            post_tokens = [lemmatizer.lemmatize(w, 'n') for w in post_tokens]
            post_tokens = [lemmatizer.lemmatize(w, 'v') for w in post_tokens]
            title_tokens = [lemmatizer.lemmatize(w, 'n') for w in title_tokens]
            title_tokens = [lemmatizer.lemmatize(w, 'v') for w in title_tokens]
            csv_cols.append({'author': row['author'],
                             'selftext': post_tokens,
                             'title': title_tokens})
            # TODO need to collect frequencies of words in the entire corpus
            # TODO update frequencies mapping from word->count and also get a sum
    return csv_cols, frequencies

[nltk_data] Downloading package wordnet to /Users/apple/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
parsed = parse_reddit_csv('final_proj_data_preprocessed_1000sample.csv')

Reading from final_proj_data_preprocessed_1000sample.csv


In [55]:
counts = Counter(parsed[0][0]['selftext']).items()
freq = pd.DataFrame(counts)
#inverse relative frequency
freq['inv_rfreq'] = freq[1].sum()/freq[1]

In [59]:
for w in freq[0].to_list():
    emb = model[freq[0].to_list()]
except KeyError:

SyntaxError: unexpected EOF while parsing (<ipython-input-59-e0093e442dd1>, line 3)

In [52]:
freq[0]

0              trip
1              What
2             today
3             alone
4              door
5              Best
6     accommodation
7            people
8     theoretically
9              hour
10            sleep
11            throw
12              350
13              way
14           accord
15              one
16           groggy
17            route
18            Hello
19            visit
20              cut
21          holiday
22              pre
23             also
24                3
25            house
26            might
27              But
28              cop
29             able
          ...      
48            since
49             bang
50           anyway
51               km
52           replan
53               go
54    Unfortunately
55            drive
56             rest
57            gonna
58           either
59           lovely
60       reschedule
61             road
62          drunken
63                I
64            place
65            order
66         possible
